In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/csv/anonymized_sales_data.csv')

## Overview

In [3]:
df.describe(include='all')

,invoice_date,customer_code,location_code,channel_text,customer_status,conventional_synthetic,variety,size,sale_value
count,541796,541796.000000,541796,541796,541796,541796,541796,541796,541796.000000
unique,1414,NaN,7,6,1,2,4,7,NaN
top,2021-09-07,NaN,TX_IAH,Manufacturer/Distributor,Active,CONVENTIONAL,5W30,Case_12X2_Gallon,NaN
freq,646,NaN,233794,368473,541796,531016,298737,139854,NaN
mean,NaN,203148.639268,NaN,NaN,NaN,NaN,NaN,NaN,1372.337826
std,NaN,192759.723394,NaN,NaN,NaN,NaN,NaN,NaN,2694.498508
min,NaN,3409.000000,NaN,NaN,NaN,NaN,NaN,NaN,-12924.910000
25%,NaN,14793.000000,NaN,NaN,NaN,NaN,NaN,NaN,563.490000
50%,NaN,132281.000000,NaN,NaN,NaN,NaN,NaN,NaN,696.350000
75%,NaN,403912.000000,NaN,NaN,NaN,NaN,NaN,NaN,1145.830000


## Changes

### customer_status

### invoice_date

In [4]:
# Convert to datetime object
df['invoice_date'] = pd.to_datetime(df['invoice_date'], format='%Y-%m-%d')

### sale_value

In [5]:
df['sale_value'] = df['sale_value'].astype(np.int32)

### remove other columns

We remove the other categories such as variety because the sales are to be aggregated. This is mainly because there is highly imbalanced data in terms of these categories. Location is left so that the data can be split by location, but is removed below

In [6]:
to_delete = set(df.columns) - set(['invoice_date', 'sale_value', 'location_code'])
df = df.drop(columns=to_delete)

In [7]:
df.dtypes

invoice_date     datetime64[ns]
location_code            object
sale_value                int32
dtype: object

### drop negative sales

In [8]:
df.shape

(541796, 3)

In [9]:
df = df[df['sale_value'] >= 0]
df.shape

(541789, 3)

### aggregate sales data and fill in missing dates

In [10]:
dft = df.groupby('invoice_date').agg({'sale_value': 'sum'}).reset_index()
all_dates = pd.date_range(start=dft.invoice_date.min(), end = dft.invoice_date.max())
dft = dft.set_index('invoice_date').reindex(all_dates, fill_value=np.nan)
dft.sale_value = dft.sale_value.interpolate()
dft.index.name = 'invoice_date'
dft

,sale_value
invoice_date,
2018-01-01,222537
2018-01-02,548741
2018-01-03,643849
2018-01-04,496851
2018-01-05,451184
...,...
2021-11-10,899971
2021-11-11,671045
2021-11-12,428742


### split by location

In [11]:
locs = list(df['location_code'].unique())
locs

['TX_AUS', 'TX_IAH', 'TX_SAT', 'NC_CLT', 'TX_DFW', 'OK_TUL', 'CA_SMF']

In [12]:
%%timeit

if not os.path.isdir('data'):
    os.mkdir('data')
    os.mkdir('data/csv')

for loc in locs:
    path = os.path.join('data', loc) + '.parquet'
    csv_path = os.path.join('data', 'csv', loc + '.csv')
    write_df = df.query(f'location_code == \'{loc}\'').drop(columns='location_code')

    write_df.to_parquet(path)
    write_df.to_csv(csv_path)    

1.02 s ± 7.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### benchmarking

In [13]:
%%timeit
# Slower and ...
for loc in ['TX_AUS', 'TX_IAH', 'TX_SAT', 'NC_CLT', 'TX_DFW', 'OK_TUL', 'CA_SMF']:
    _ = pd.read_csv(os.path.join('data', 'csv', loc + '.csv'))

79.2 ms ± 840 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
# ...does not preserve data column types
pd.read_csv('data/csv/TX_AUS.csv').dtypes

Unnamed: 0       int64
invoice_date    object
sale_value       int64
dtype: object

In [15]:
%%timeit
for loc in ['TX_AUS', 'TX_IAH', 'TX_SAT', 'NC_CLT', 'TX_DFW', 'OK_TUL', 'CA_SMF']:
    _ = pd.read_parquet(os.path.join('data', loc + '.parquet'))

23.1 ms ± 63.5 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
pd.read_parquet('data/TX_AUS.parquet').dtypes

invoice_date    datetime64[ns]
sale_value               int32
dtype: object